# Llama Stack Agent Workflows

This notebook contains minimal implementations of common agent workflows discussed in the blog post [Building Effective Agent Workflows](https://www.anthropic.com/research/building-effective-agents). Examples are copied from [anthropic-cookbook](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents). 

In [1]:
MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"

In [8]:
from llama_stack_client import LlamaStackClient
from llama_stack_client.types.agent_create_params import AgentConfig
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.agent import Agent
from rich.pretty import pprint
import json
import uuid

client = LlamaStackClient(base_url="http://localhost:8321")

## 1. Basic Multi-LLM Workflow

1. **Prompt Chaining**: Decomposes a task into sequential subtasks, where each step builds on previous results. 

2. **Parallelization**: Distributes independent subtasks acorss multiple LLMs for concurrent processing. 

3. **Routing**: Dynamically selects specialized LLM paths based on input characteristics. 

#### 1.1 Prompt Chaining

In [19]:
vanilla_agent_config = AgentConfig(
    model=MODEL_ID,
    instructions="""
    You are a helpful assistant capable of structuring data extraction and formatting. 

    You will be given tasks to extract and format data from a performance report. Here is the report:

    Q3 Performance Summary:
    Our customer satisfaction score rose to 92 points this quarter.
    Revenue grew by 45% compared to last year.
    Market share is now at 23% in our primary market.
    Customer churn decreased to 5% from 8%.
    New user acquisition cost is $43 per user.
    Product adoption rate increased to 78%.
    Employee satisfaction is at 87 points.
    Operating margin improved to 34%.
    """,
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
    },
    toolgroups=[],
    tool_config={
        "tool_choice": "auto",
        "tool_prompt_format": "python_list",
    },
    input_shields=[],
    output_shields=[],
    enable_session_persistence=False,
)

vanilla_agent = Agent(client, vanilla_agent_config)

session_id_1 = vanilla_agent.create_session(session_name=f"vanilla_agent_{uuid.uuid4()}")

prompts = [
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth""",

    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth""",

    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction""",

    """Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |""",
]

for i, prompt in enumerate(prompts):    
    response = vanilla_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id_1,
        stream=False,
    )
    print("========= Turn: ", i, "=========")
    print(response.output_message.content)
    print("\n")

========= Turn:  0 =========
92: customer satisfaction score
45%: revenue growth
23%: market share
5%: customer churn
43: new user acquisition cost
78%: product adoption rate
87: employee satisfaction
34%: operating margin
8%: customer churn (changed to reflect the original value for comparison)


========= Turn:  1 =========
92%: customer satisfaction
45%: revenue growth
23%: market share
5%: customer churn
0.043: new user acquisition cost
78%: product adoption rate
87%: employee satisfaction
34%: operating margin
8%: previous customer churn


========= Turn:  2 =========
92%: customer satisfaction
87%: employee satisfaction
78%: product adoption rate
45%: revenue growth
34%: operating margin
23%: market share
8%: previous customer churn
5%: customer churn
0.043: new user acquisition cost


========= Turn:  3 =========
| Metric | Value |
|:--|--:|
| Customer Satisfaction | 92% |
| Employee Satisfaction | 87% |
| Product Adoption Rate | 78% |
| Revenue Growth | 45% |
| Operating Margin